# Overview

This a notebook that inspects the results of a WarpX simulation and a Warp simulation at the same time. Typically used to compare the results of both codes for the same set of parameters.

# Instructions

- Enter the paths for the WarpX and the Warp simulations
- Execute the cells below one by one, by selecting them with your mouse and typing `Shift + Enter`

In [1]:
# Import statements
import sys
from tqdm import tqdm
import yt, glob
yt.funcs.mylog.setLevel(50)
from IPython.display import clear_output
import numpy as np
from ipywidgets import interact, RadioButtons, IntSlider
from opmd_viewer import OpenPMDTimeSeries
from yt.units import volt
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
# Define path way for WarpX results and find iterations
path_warpx = '/Users/mthevenet/warpX_directory/res_warpx/valid_pwfa/'
file_list_warpx = glob.glob(path_warpx + 'plt?????')
iterations_warpx = [ int(file_name[len(file_name)-5:]) for file_name in file_list_warpx ]

In [3]:
# Define path way for Warp results and find iterations
path_warp = '/Users/mthevenet/warp_results/valid_pwfa/'
file_list_warp = glob.glob(path_warp + 'diags/hdf5/data????????.h5')
iterations_warp = [ int(file_name[len(file_name)-11:len(file_name)-3]) for file_name in file_list_warp ]

# Functions to plot the fields

Note that the data are loaded with yt and plotted using matplotlib

In [4]:
def plot_field( iteration, field, slicing_direction='y'):
    
    # First dataset
    ds = yt.load( path_warpx + '/plt%05d/' %iteration )
    all_data_level_0 = ds.covering_grid(level=0, 
                        left_edge=ds.domain_left_edge, dims=ds.domain_dimensions)
  
    # Second dataset
#     ts = OpenPMDTimeSeries(path_warp + 'diags/hdf5/')    
    ds2 = yt.load( path_warp + 'diags/hdf5/data%08d.h5' %iteration )
    all_data_level_02 = ds2.covering_grid(level=0, 
                        left_edge=ds2.domain_left_edge, dims=ds2.domain_dimensions)

    # first dataset loaded via yt
    left_edge = ds.domain_left_edge.convert_to_mks()*1e6
    right_edge = ds.domain_right_edge.convert_to_mks()*1e6
    if slicing_direction == 'x':
        n = int( ds.domain_dimensions[0]//2 )
        data2d = all_data_level_0[field][n, :, :]
        extent = [ left_edge[2], right_edge[2], left_edge[1], right_edge[1] ]
    elif slicing_direction == 'y':
        n = int( ds.domain_dimensions[1]//2 )
        data2d = all_data_level_0[field][:, n, :]
        extent = [ left_edge[2], right_edge[2], left_edge[0], right_edge[0] ]
    elif slicing_direction == 'z':
        n = int( ds.domain_dimensions[2]//2 )
        data2d = all_data_level_0[field][:, :, n]
        extent = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]

    # second dataset loaded via yt
    left_edge = ds2.domain_left_edge.convert_to_mks()*1e6
    right_edge = ds2.domain_right_edge.convert_to_mks()*1e6
    field_reformat = field[0].upper() + '_' + field[1]
    if slicing_direction == 'x':
        n = int( ds2.domain_dimensions[0]//2 )
        data2d2 = all_data_level_02[field_reformat][n, :, :]
        extent2 = [ left_edge[2], right_edge[2], left_edge[1], right_edge[1] ]
    elif slicing_direction == 'y':
        n = int( ds2.domain_dimensions[1]//2 )
        data2d2 = all_data_level_02[field_reformat][:, n, :]
        extent2 = [ left_edge[2], right_edge[2], left_edge[0], right_edge[0] ]
    elif slicing_direction == 'z':
        n = int( ds2.domain_dimensions[2]//2 )
        data2d2 = all_data_level_02[field_reformat][:, :, n]
        extent2 = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]

    if slicing_direction == 'x':
        yaxis_label = 'y'
    if slicing_direction == 'y':
        yaxis_label = 'x'
    if slicing_direction == 'z':
        yaxis_label = 'y'

#         xlim = [5,25]
#         ylim = [-20,20]

    plt.clf()
    # first dataset
    plt.subplot(2,1,1)
    plt.title("WarpX %s at iteration %d" %(field, iteration) )
    plt.imshow( data2d, interpolation='nearest', cmap='viridis',
              origin='lower', extent=extent, aspect='auto')
    plt.ylabel(yaxis_label + ' (um)')
    vmin, vmax = plt.gci().get_clim()
    plt.colorbar()
    if 'xlim' in locals():
        plt.xlim(xlim)
    if 'ylim' in locals():
        plt.ylim(ylim)
    plt.xticks([])
    
    # second dataset
    plt.subplot(2,1,2)
    plt.title("Warp %s at iteration %d" %(field, iteration) )
    plt.imshow( data2d2, interpolation='nearest', cmap='viridis',
              origin='lower', extent=extent2, aspect='auto',vmin=vmin,vmax=vmax)
    plt.colorbar()
    if 'xlim' in locals():
        plt.xlim(xlim)
    if 'ylim' in locals():
        plt.ylim(ylim)
    plt.xlabel('z (um)')
    plt.ylabel(yaxis_label + ' (um)')

# Interactive viewer

In [5]:
iterations = iterations_warp
interact(plot_field, 
         iteration = IntSlider(min=min(iterations), max=max(iterations), step=iterations[1]-iterations[0]),
         field = RadioButtons( options=['jx', 'jy', 'jz', 'Ex', 'Ey', 'Ez'], value='jx'),
         slicing_direction = RadioButtons( options=[ 'x', 'y', 'z'], value='x'))